# Operaciones avanzadas con DataFrames

## Descripción de las variables

El dataset, obtenido de <a target = "_blank" href="https://www.transtats.bts.gov/Fields.asp?Table_ID=236">este link</a> está compuesto por las siguientes variables referidas siempre al año 2018:

1. **Month** 1-4
2. **DayofMonth** 1-31
3. **DayOfWeek** 1 (Monday) - 7 (Sunday)
4. **FlightDate** fecha del vuelo
5. **Origin** código IATA del aeropuerto de origen
6. **OriginCity** ciudad donde está el aeropuerto de origen
7. **Dest** código IATA del aeropuerto de destino
8. **DestCity** ciudad donde está el aeropuerto de destino  
9. **DepTime** hora real de salida (local, hhmm)
10. **DepDelay** retraso a la salida, en minutos
11. **ArrTime** hora real de llegada (local, hhmm)
12. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
13. **Cancelled** si el vuelo fue cancelado (1 = sí, 0 = no)
14. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
15. **Diverted** si el vuelo ha sido desviado (1 = sí, 0 = no)
16. **ActualElapsedTime** tiempo real invertido en el vuelo
17. **AirTime** en minutos
18. **Distance** en millas
19. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
20. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
21. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
22. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
23. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

# Leemos los datos y quitamos filas con NA y convertimos a numéricas las columnas inferidas incorrectamente
flightsDF = spark.read.option("header", "true").option("inferSchema", "true").csv("dbfs:/FileStore/shared_uploads/paula.roman@bosonit.com/flights_jan_apr_2018.csv")

# Convertimos a enteros y re-categorizamos ArrDelay en una nueva columna ArrDelayCat
# None (< 15 min), Slight(entre 15 y 60 min), Huge (> 60 min)

cleanFlightsDF = flightsDF.withColumn("ArrDelayCat", F.when(F.col("ArrDelay") < 15, "None")\
                                                      .when((F.col("ArrDelay") >= 15) & (F.col("ArrDelay") < 60), "Slight")\
                                                      .otherwise("Huge"))\
                           .cache()

## Hagamos algunas preguntas a los datos para obtener conclusiones

Imaginemos que somos los dueños de una web de viajes que rastrea internet en busca de vuelos en agencias y otras páginas, los compara y recomienda el más adecuado para el aeropuerto. Junto con esta recomendación, querríamos dar también información sobre vuelos fiables y no fiables en lo que respecta a la puntualidad. Esto depende de muchos factores, como el origen y destino, duración del vuelo, hora del día, etc.

### Agrupación y agregaciones

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Cuáles son los vuelos (origen, destino) con mayor retraso medio? ¿Cuántos vuelos existen entre cada par de aeropuertos?</p>
<p><b>PISTA</b>: Tras hacer las agregaciones para cada pareja "Origin", "Dest" (una agregación para el retraso medio y otra para contar), aplica el método sort(F.col("avgDelay").desc()) para ordenar de forma decreciente por la nueva columna del retraso medio.
</div>

In [0]:
FlightsDelayedDF = cleanFlightsDF.groupBy("Origin","Dest").agg(F.mean("ArrDelay").alias("avgDelay"), F.count("*").alias("count")).sort(F.col("avgDelay").desc())
FlightsDelayedDF.show()

+------+----+------------------+-----+
|Origin|Dest|          avgDelay|count|
+------+----+------------------+-----+
|   RDM| MFR|            1347.0|    2|
|   MDT| HPN|             798.0|    1|
|   ORD| GTF|             212.0|    1|
|   ICT| DAY|             210.0|    1|
|   ELM| ATL|             169.0|    2|
|   DSM| PIA|             168.0|    1|
|   ERI| ITH|             160.0|    1|
|   YNG| PIE|             141.0|    1|
|   CMH| HOU|             120.0|    1|
|   HRL| DAL|             111.0|    1|
|   PPG| HNL|109.85714285714286|   35|
|   HNL| PPG|105.85714285714286|   35|
|   PIE| YNG|             104.0|    1|
|   AVP| SFB|              93.0|    1|
|   ACY| MSY| 87.45454545454545|   11|
|   CPR| LAS|              85.0|    1|
|   LAS| CPR|              82.0|    1|
|   TTN| BNA|              76.5|   10|
|   MSP| PVD|              74.0|    1|
|   TUL| OKC|              69.0|    1|
+------+----+------------------+-----+
only showing top 20 rows



<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Es el avión un medio de transporte fiable? Mostrar el número de vuelos en cada categoría de retraso.</p>
En lugar de llamar agg(F.count("*")), podemos llamar a la transformación count() sobre el resultado de groupBy(), y creará
automáticamente una columna llamada "count" con los conteos para cada grupo.
<p> Ahora agrupar también por cada aeropuerto de origen, y mostrando una columna distinta por cada tipo de retraso, con el recuento. PISTA: utilizar la función pivot("colName").</p>

In [0]:
pivot = cleanFlightsDF.groupBy("Origin").pivot("ArrDelayCat").agg(F.count("*").alias("Conteo"), F.max("ArrDelay").alias("MaxArrDelay")).sort("Origin")
pivot.show()

+------+-----------+----------------+-----------+----------------+-------------+------------------+
|Origin|Huge_Conteo|Huge_MaxArrDelay|None_Conteo|None_MaxArrDelay|Slight_Conteo|Slight_MaxArrDelay|
+------+-----------+----------------+-----------+----------------+-------------+------------------+
|   ABE|        193|           674.0|       1230|            14.0|          239|              59.0|
|   ABI|         56|           397.0|        497|            14.0|           76|              59.0|
|   ABQ|        457|          1650.0|       7184|            14.0|          947|              59.0|
|   ABR|         22|           584.0|        198|            13.0|           20|              40.0|
|   ABY|         38|           641.0|        239|            14.0|           57|              50.0|
|   ACT|         40|           500.0|        386|            14.0|           42|              59.0|
|   ACV|         40|           900.0|        291|            14.0|           33|              59.0|


<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Hay relación entre el día de la semana y el retraso a la salida o a la llegada?</p>
    <p><b>PISTA</b>: Calcula el retraso medio a la salida y a la llegada para cada día de la semana y ordena por una de ellas descendentemente.</p>
    <p> Ahora haz lo mismo para cada día pero solo con el retraso a la llegada, desagregado por cada aeropuerto de salida, utilizando la función pivot(). </p>
</div>

<div class="alert alert-block alert-info">
<p><b>LA FUNCIÓN PIVOT</b>: Puede ser interesante ver, para cada (Origin, Dest), el retraso promedio por
día de la semana. Si agrupamos por esas tres variables (Origin, Dest, DayOfWeek), nuestro resultado tendría demasiadas filas para ser fácil de visualizar (7 x 1009 ya que hay 1009 combinaciones de (Origin, DayOfWeek)). En cambio, vamos a crear 7 columnas, una por día de la semana, en nuestro resultado DF. Lo haremos utilizando una de las variables de agrupación (DayOfWeek) como <i> variable pivot</i>. Como esta variable tiene 7 valores distintos, se crearán 7 columnas nuevas. De esta manera, visualizaremos toda la información de cada combinación (Origen, Dest) condensada en una fila con 7 columnas con los 7 retrasos promedio correspondientes a ese (Origen, Dest) en cada día de la semana.
</div>

In [0]:
avgDelaysDF = flightsDF.groupBy("DayOfWeek").agg(F.mean("ArrDelay").alias("avgArrDelay"), F.mean("DepDelay").alias("avgDepDelay")).sort("DayOfWeek")
avgDelaysDF.show()

+---------+-------------------+------------------+
|DayOfWeek|        avgArrDelay|       avgDepDelay|
+---------+-------------------+------------------+
|        1|  5.391113068725289|10.430177708665964|
|        2| 2.8412409647873806| 8.246502522185226|
|        3| 3.0525338339576717|  8.47071347600168|
|        4| 2.7390527404801026|  8.35856546210902|
|        5|  5.027363815430113|10.220785437977693|
|        6|-0.5748593305876211| 6.278199328016013|
|        7| 3.2344449424598207| 9.142161259888235|
+---------+-------------------+------------------+



In [0]:
avgArrDelaysDF = flightsDF.groupby("Origin").pivot("DayOfWeek").agg(F.mean("ArrDelay").alias("MeanArrDelay")).sort("Origin")

avgArrDelaysDF.show()

+------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+-------------------+
|Origin|                  1|                   2|                  3|                  4|                   5|                  6|                  7|
+------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+-------------------+
|   ABE|  14.95330739299611|  13.893518518518519| 15.276785714285714|   5.92436974789916|  18.183333333333334|  6.940298507462686| 10.027149321266968|
|   ABI|              10.65|   16.36470588235294|-0.5476190476190477| 0.3763440860215054|   4.641304347826087|  16.49206349206349| 4.6063829787234045|
|   ABQ| 1.4424379232505644|  0.3634064080944351| 1.7364085667215816|  3.111111111111111|-0.17689243027888446|-1.6557213930348258|-0.2520193861066236|
|   ABR| 10.571428571428571|            12.46875|  6.294117647058823| 16.441176470588236|   8.

### Operaciones JOIN y de ventana

Estaría bien tener el retraso promedio de una ruta junto a cada vuelo, para que podamos ver qué vuelos tuvieron un retraso que fue superior o inferior al retraso promedio de esa ruta.

<div class="alert alert-block alert-success">
    <b> PREGUNTA </b>:
Usa el averageDelayOriginDestDF creado anteriormente, elimina la columna de conteo y luego únerlo con cleanFlightsDF, utilizando Origin y Dest como columnas de enlace. Finalmente, selecciona solo las columnas Origin, Dest, DayOfWeek, ArrDelay y avgDelay del resultado.
</div>

In [0]:
averageDelayOriginDestDF = cleanFlightsDF.groupBy("Origin", "Dest")\
                                         .agg(F.mean("ArrDelay").alias("avgDelay"))

joinedDF = cleanFlightsDF.join(averageDelayOriginDestDF, 
                               on = ["Origin", "Dest"], 
                               how = "left_outer")\
                         .select("Origin", "Dest", "DayOfWeek", "ArrDelay", "avgDelay")

joinedDF.show()

+------+----+---------+--------+------------------+
|Origin|Dest|DayOfWeek|ArrDelay|          avgDelay|
+------+----+---------+--------+------------------+
|   SYR| DTW|        7|    null|0.4039408866995074|
|   SYR| LGA|        3|   -13.0| 7.643799472295514|
|   SYR| LGA|        6|    24.0| 7.643799472295514|
|   SYR| LGA|        7|   -24.0| 7.643799472295514|
|   SYR| LGA|        1|   -10.0| 7.643799472295514|
|   SYR| LGA|        2|   -15.0| 7.643799472295514|
|   SYR| LGA|        3|   -24.0| 7.643799472295514|
|   SYR| LGA|        4|   -27.0| 7.643799472295514|
|   SYR| LGA|        5|   149.0| 7.643799472295514|
|   SYR| LGA|        6|    -5.0| 7.643799472295514|
|   SYR| LGA|        1|   -23.0| 7.643799472295514|
|   SYR| LGA|        2|    -8.0| 7.643799472295514|
|   SYR| LGA|        3|    63.0| 7.643799472295514|
|   SYR| LGA|        4|   -10.0| 7.643799472295514|
|   SYR| LGA|        5|   -30.0| 7.643799472295514|
|   SYR| LGA|        6|    14.0| 7.643799472295514|
|   SYR| LGA

<div class="alert alert-block alert-info">
    <p><b>BONUS (OPCIONAL)</b>: crear una nueva columna <i>belowAverage</i> que tenga valor True si ArrDelay es menor que el avgDelay de esa ruta, y False en caso contrario. No utilizar la función when() sino el operador de comparación directamente entre columnas, la cual devolverá una columna booleana.
</div>

In [0]:
cleanFlightsNewDF = joinedDF.withColumn("belowAverage", F.when(F.col("ArrDelay") < F.col("avgDelay"), "True").otherwise("False"))

cleanFlightsNewDF.show()

+------+----+---------+--------+------------------+------------+
|Origin|Dest|DayOfWeek|ArrDelay|          avgDelay|belowAverage|
+------+----+---------+--------+------------------+------------+
|   SYR| DTW|        7|    null|0.4039408866995074|       False|
|   SYR| LGA|        3|   -13.0| 7.643799472295514|        True|
|   SYR| LGA|        6|    24.0| 7.643799472295514|       False|
|   SYR| LGA|        7|   -24.0| 7.643799472295514|        True|
|   SYR| LGA|        1|   -10.0| 7.643799472295514|        True|
|   SYR| LGA|        2|   -15.0| 7.643799472295514|        True|
|   SYR| LGA|        3|   -24.0| 7.643799472295514|        True|
|   SYR| LGA|        4|   -27.0| 7.643799472295514|        True|
|   SYR| LGA|        5|   149.0| 7.643799472295514|       False|
|   SYR| LGA|        6|    -5.0| 7.643799472295514|        True|
|   SYR| LGA|        1|   -23.0| 7.643799472295514|        True|
|   SYR| LGA|        2|    -8.0| 7.643799472295514|        True|
|   SYR| LGA|        3|  

**PREGUNTA**: repetir la operación utilizando funciones de ventana, sin usar `join`.

In [0]:
from pyspark.sql import Window
w = Window().partitionBy("Origin", "Dest")

withAvgDF = cleanFlightsDF.withColumn("avgDelay", F.mean("ArrDelay").over(w))
withAvgDF.where("Origin = 'ATL' and Dest = 'GSP'")\
         .select("Origin", "Dest", "DayOfWeek", "ArrDelay", "avgDelay").show()

+------+----+---------+--------+------------------+
|Origin|Dest|DayOfWeek|ArrDelay|          avgDelay|
+------+----+---------+--------+------------------+
|   ATL| GSP|        3|    -6.0|-1.734910277324633|
|   ATL| GSP|        3|    -3.0|-1.734910277324633|
|   ATL| GSP|        3|    47.0|-1.734910277324633|
|   ATL| GSP|        4|    null|-1.734910277324633|
|   ATL| GSP|        4|    -6.0|-1.734910277324633|
|   ATL| GSP|        4|    38.0|-1.734910277324633|
|   ATL| GSP|        1|   -13.0|-1.734910277324633|
|   ATL| GSP|        1|   -15.0|-1.734910277324633|
|   ATL| GSP|        1|    10.0|-1.734910277324633|
|   ATL| GSP|        5|    -6.0|-1.734910277324633|
|   ATL| GSP|        5|    -3.0|-1.734910277324633|
|   ATL| GSP|        5|   -14.0|-1.734910277324633|
|   ATL| GSP|        1|    10.0|-1.734910277324633|
|   ATL| GSP|        1|    14.0|-1.734910277324633|
|   ATL| GSP|        1|    -3.0|-1.734910277324633|
|   ATL| GSP|        2|   -22.0|-1.734910277324633|
|   ATL| GSP

<div class="alert alert-block alert-success">
<b> PREGUNTA </b>: Vamos a construir otro DF con información sobre los aeropuertos (en una situación real, tendríamos otra tabla en la base de datos como la tabla de la entidad Aeropuerto). Sin embargo, solo tenemos información sobre algunos aeropuertos. Nos gustaría agregar esta información a cleanFlightsDF como nuevas columnas, teniendo en cuenta que queremos que la información del aeropuerto coincida con el aeropuerto de origen de flightsDF. Utilizar la operación de unión adecuada para asegurarse de que no se perderá ninguna de las filas existentes de cleanFlightsDF después de la unión.
</div>

In [0]:
airportsDF = spark.createDataFrame([
    ("JFK", "John F. Kennedy International Airport", 1948),
    ("LIT", "Little Rock National Airport", 1931),
    ("SEA", "Seattle-Tacoma International Airport", 1949),
], ["IATA", "FullName", "Year"])

In [0]:
joinedFlightsDF = cleanFlightsDF.join(airportsDF, on = (cleanFlightsDF["Origin"] == airportsDF["IATA"]), how = "left_outer").select("FlightDate" ,"Origin", "OriginCity", "FullName", "Year")

# PREGUNTA: mostrar algunas filas donde FullName no sea null
joinedFlightsDF.where(F.col("FullName").isNotNull()).show()

+-------------------+------+-----------+--------------------+----+
|         FlightDate|Origin| OriginCity|            FullName|Year|
+-------------------+------+-----------+--------------------+----+
|2018-01-01 00:00:00|   SEA|Seattle, WA|Seattle-Tacoma In...|1949|
|2018-01-01 00:00:00|   SEA|Seattle, WA|Seattle-Tacoma In...|1949|
|2018-01-01 00:00:00|   SEA|Seattle, WA|Seattle-Tacoma In...|1949|
|2018-01-01 00:00:00|   SEA|Seattle, WA|Seattle-Tacoma In...|1949|
|2018-01-01 00:00:00|   SEA|Seattle, WA|Seattle-Tacoma In...|1949|
|2018-01-02 00:00:00|   SEA|Seattle, WA|Seattle-Tacoma In...|1949|
|2018-01-02 00:00:00|   SEA|Seattle, WA|Seattle-Tacoma In...|1949|
|2018-01-02 00:00:00|   SEA|Seattle, WA|Seattle-Tacoma In...|1949|
|2018-01-02 00:00:00|   SEA|Seattle, WA|Seattle-Tacoma In...|1949|
|2018-01-02 00:00:00|   SEA|Seattle, WA|Seattle-Tacoma In...|1949|
|2018-01-03 00:00:00|   SEA|Seattle, WA|Seattle-Tacoma In...|1949|
|2018-01-03 00:00:00|   SEA|Seattle, WA|Seattle-Tacoma In...|1

## User-defined functions (UDFs)

Vamos a construir un UDF para convertir millas a kilómetros. Ten en cuenta que esto podría hacerse fácilmente multiplicando directamente la columna de millas por 1.6 (y sería mucho más eficiente), ya que Spark permite el producto entre una columna y un número. En todos los casos en los que Spark proporciona funciones integradas para realizar una tarea (como esta), debes usar esas funciones y no una UDF. Las UDF deben emplearse solo cuando no hay otra opción.

La razón es que las funciones integradas de Spark están optimizadas y Catalyst, el optimizador automático de código integrado en Spark, puede optimizarlo aún más. Sin embargo, las UDF son una caja negra para Catalyst y su contenido no se optimizará, y por lo tanto, generalmente son mucho más lentas.

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# Primer paso: crear una función de Python que reciba UN número y lo multiplique por 1.6
def milesToKm(miles):
    return miles*1.6

# Vamos a probarla
print(milesToKm(5)) # 5 millas a km: 8 km

# Segundo paso: crear un objeto UDF que envuelva a nuestra función. 
# Hay que especificar el tipo de dato que devuelve nuestra función
udfMilesToKm = F.udf(milesToKm, DoubleType())

# Con esto, Spark será capaz de llamar a nuestra función milesToKm sobre cada uno de los valores de una columna numérica.
# Spark enviará el código de nuestra función a los executors a través de la red, y cada executor la ejecutará sobre las
# particiones (una por una) que estén en ese executor

# Tercer paso: vamos a probar la UDF añadiendo una nueva columna con el resultado de la conversión
flightsWithKm = cleanFlightsDF.withColumn("DistKm", udfMilesToKm(F.col("Distance")))

flightsWithKm.select("Origin", "Dest", "Distance", "DistKM")\
             .distinct()\
             .show(5)

8.0
+------+----+--------+------------------+
|Origin|Dest|Distance|            DistKM|
+------+----+--------+------------------+
|   SYR| DTW|   374.0|             598.4|
|   DTW| PHL|   453.0| 724.8000000000001|
|   SYR| LGA|   198.0|             316.8|
|   TVC| DTW|   207.0|331.20000000000005|
|   HPN| ATL|   780.0|            1248.0|
+------+----+--------+------------------+
only showing top 5 rows



<div class="alert alert-block alert-info">
<p><b>BONUS</b>: Crea tu propia UDF que convierta DayOfWeek en una cadena.
Puedes hacerlo creando una función de Python que reciba un número entero y devuelva el día de la semana,
simplemente leyendo desde un vector de cadenas de longitud 7 el valor en la posición indicada por el argumento entero. Para la UDF, recuerda que tu función devuelve un StringType(). Finalmente, prueba tu UDF creando una nueva columna "DayOfWeekString".
</div>

In [0]:
from pyspark.sql.types import StringType

# Primer paso: creamos una función de python que convierte un número entero en el día de la semana como cadena
def dayOfWeekToString(dayInteger):
    # En nuestros datos Monday es 1 pero las listas de python empiezan en el 0 y 
    # queremos usar el dayInteger como índice del vector
    daysOfWeek = ["", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    dia = daysOfWeek[dayInteger]
    return dia

print(dayOfWeekToString(5))
    
# Segundo paso: ajustamos nuestra función con un Spark UDF para que Spark pueda invocarlo en cada valor de una columna completa
# De esta manera, Spark puede enviar nuestra función a los ejecutores, que eventualmente ejecutarán la función en las particiones
# de los datos que tiene cada ejecutor
dayOfWeekStringUDF = F.udf(dayOfWeekToString, StringType())

# Tercer paso: intentemos nuestro UDF agregando una nueva columna que resulta de transformar (a través del UDF) el
# columna existente DayOfWeek
flightsWithDayOfWeekStr = cleanFlightsDF.withColumn("DayOfWeekString", dayOfWeekStringUDF(F.col("DayOfWeek")))

flightsWithDayOfWeekStr.select("Origin", "Dest", "DayOfWeek", "DayOfWeekString").distinct().show()

Friday
+------+----+---------+---------------+
|Origin|Dest|DayOfWeek|DayOfWeekString|
+------+----+---------+---------------+
|   OMA| DTW|        1|         Monday|
|   DTW| BWI|        2|        Tuesday|
|   BWI| DTW|        2|        Tuesday|
|   HPN| ATL|        6|       Saturday|
|   MSP| CVG|        2|        Tuesday|
|   SYR| LGA|        5|         Friday|
|   SYR| LGA|        1|         Monday|
|   BOS| BUF|        1|         Monday|
|   BOS| BUF|        5|         Friday|
|   SYR| LGA|        7|         Sunday|
|   SYR| LGA|        2|        Tuesday|
|   DTW| PHL|        7|         Sunday|
|   SYR| LGA|        3|      Wednesday|
|   BOS| BUF|        7|         Sunday|
|   BOS| BUF|        2|        Tuesday|
|   SYR| LGA|        6|       Saturday|
|   TVC| DTW|        1|         Monday|
|   SYR| LGA|        4|       Thursday|
|   SYR| DTW|        7|         Sunday|
|   BOS| BUF|        3|      Wednesday|
+------+----+---------+---------------+
only showing top 20 rows

